In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime

In [2]:
agencies = pd.read_json('../data/launch-library/agencies.json', encoding='utf=8')

In [3]:
agencies.dtypes

id                                   int64
url                                 object
name                                object
featured                              bool
type                                object
country_code                        object
abbrev                              object
description                         object
administrator                       object
founding_year                      float64
launchers                           object
spacecraft                          object
parent                              object
launch_library_url                  object
total_launch_count                   int64
successful_launches                  int64
consecutive_successful_launches      int64
failed_launches                      int64
pending_launches                     int64
successful_landings                  int64
failed_landings                      int64
attempted_landings                   int64
consecutive_successful_landings      int64
info_url   

In [4]:
spaceX_founding = datetime(2002, 3, 14)
spaceX_first_launch = datetime(2008, 9, 28)
spaceX_first_test_landing = datetime(2013, 9, 29)
spaceX_first_soft_touch_down = datetime(2014, 4, 8)
spaceX_first_landing = datetime(2015, 12, 21)

In [5]:
agencies_with_founding_year = agencies[agencies['founding_year'].notna()].sort_values('founding_year')

Group by `founding_year` and calculate total agencies of all categories for each year including overall total

In [6]:
total_agencies_per_year = []

for year in agencies_with_founding_year['founding_year'].unique():
  current_year = agencies_with_founding_year[agencies_with_founding_year['founding_year'] == year]
  new = current_year.count()['id']
  new_government = current_year[current_year['type'] == 'Government'].count()['id']
  new_commercial = current_year[current_year['type'] == 'Commercial'].count()['id']
  new_private = current_year[current_year['type'] == 'Private'].count()['id']
  new_multinational = current_year[current_year['type'] == 'Multinational'].count()['id']
  
  new_entry = {
    'year': year,
    'new': new,
    'new_government': new_government,
    'new_commercial': new_commercial,
    'new_private': new_private,
    'new_multinational': new_multinational,
  }
  if len(total_agencies_per_year) > 0:
    new_entry['total'] = new + total_agencies_per_year[-1]['total']
    new_entry['total_government'] = new_government + total_agencies_per_year[-1]['total_government']
    new_entry['total_commercial'] = new_commercial + total_agencies_per_year[-1]['total_commercial']
    new_entry['total_private'] = new_private + total_agencies_per_year[-1]['total_private']
    new_entry['total_multinational'] = new_multinational + total_agencies_per_year[-1]['total_multinational']
  else:
    new_entry['total'] = new
    new_entry['total_government'] = new_government
    new_entry['total_commercial'] = new_commercial
    new_entry['total_private'] = new_private
    new_entry['total_multinational'] = new_multinational

  total_agencies_per_year.append(new_entry)

total_agencies_per_year = pd.DataFrame(total_agencies_per_year)

### Export to json for web display

In [7]:
total_agencies_per_year[['year', 'total_government', 'total_commercial', 'total_private']].to_json('agenciesPerYear.json')

In [8]:
x_year = total_agencies_per_year['year']
passive_color = '#bed9ec'
success_color = '#9ce68a'
commercial_color = '#1de9b6'
private_color = '#ba68c8'
annotation_bgcolor = '#222'

agencies_over_time_fig = go.Figure(
  layout={
    'template': 'plotly_dark',
    'title': 'Number of existing Space Agencies per year',
    'height': 600,
    'margin_b': 150,
    'margin_r': 150,
    'xaxis_range': [1945, 2022],
    'yaxis': {
      'side': 'right',
      'title': 'Agencies',
      'title_font_color': '#ececec',
      'title_font_size': 15,
    },
    'legend_x': 0
  },
)

agencies_over_time_fig.add_trace(go.Scatter(
  name='Total',
  mode='lines',
  hovertext=total_agencies_per_year['new'],
  x=x_year,
  y=total_agencies_per_year['total'],
  visible='legendonly'
))
agencies_over_time_fig.add_trace(go.Scatter(
  name='Governemnt',
  mode='lines',
  hovertext=total_agencies_per_year['new_government'],
  x=x_year,
  y=total_agencies_per_year['total_government']
))
agencies_over_time_fig.add_trace(go.Scatter(
  name='Commercial',
  mode='lines',
  hovertext=total_agencies_per_year['new_commercial'],
  x=x_year,
  y=total_agencies_per_year['total_commercial'],
  line={'color':commercial_color}
))
agencies_over_time_fig.add_trace(go.Scatter(
  name='Private',
  mode='lines',
  hovertext=total_agencies_per_year['new_private'],
  x=x_year,
  y=total_agencies_per_year['total_private']
))
agencies_over_time_fig.add_trace(go.Scatter(
  name='Multinational',
  mode='lines',
  hovertext=total_agencies_per_year['new_multinational'],
  x=x_year,
  y=total_agencies_per_year['total_multinational']
))

agencies_over_time_fig.add_vline(
  spaceX_founding.year,
  annotation={'text': 'SpaceX founded', 'font_color': passive_color},
  annotation_position='top',
  line={'dash': 'longdash', 'color': passive_color}
)
agencies_over_time_fig.add_vline(
  spaceX_first_launch.year,
  annotation={'text': 'SpaceX first launch', 'font_color': passive_color, 'y': 1.10},
  annotation_position='top',
  line={'dash': 'longdash', 'color': passive_color}
)
agencies_over_time_fig.add_vline(
  spaceX_first_test_landing.year,
  annotation={'text': 'SpaceX first landing test', 'font_color': passive_color, 'y': 1.05},
  annotation_position='top',
  line={'dash': 'longdash', 'color': passive_color}
)
agencies_over_time_fig.add_vline(
  spaceX_first_landing.year,
  annotation={'text': 'SpaceX first landing', 'font_color': success_color},
  annotation_position='top',
  line={'dash': 'longdash', 'color': success_color}
)

agencies_over_time_fig.add_annotation(go.layout.Annotation(
  text='The <b style="color: ' + passive_color + '">founding</b> and <b style="color: ' + passive_color + '">first successful launch</b> of<br>SpaceX were no industry changing events.',
  align='left',
  showarrow=False,
  x=2000,
  yref='paper',
  y=-0.15,
  yanchor='top',
  bgcolor=annotation_bgcolor,
  bordercolor=annotation_bgcolor,
  borderwidth=10
))
agencies_over_time_fig.add_annotation(go.layout.Annotation(
  text='However the <b style="color: ' + success_color + '">first successful landing</b> and recovery<br>of a booster rocket seems to have sparked an<br>increase in <b style="color: ' + commercial_color + '">commercial</b> and some <b style="color: ' + private_color + '">private</b> agencies.',
  align='left',
  showarrow=False,
  x=2016,
  yref='paper',
  y=-0.15,
  yanchor='top',
  bgcolor=annotation_bgcolor,
  bordercolor=annotation_bgcolor,
  borderwidth=10
))

agencies_over_time_fig.show()